In [0]:
import requests
import json
from datetime import date, timedelta

# API documentation: https://earthquake.usgs.gov/fdsnws/event/1/

In [0]:
# mount ADLS
tiers = ['bronze','silver','gold']
adls_paths = {tier: f"abfss://{tier}@earthquakedatalake.dfs.core.windows.net/" for tier in tiers}
# This creates the dictionary with the three tiers as keys and their paths as values repectively
bronze_adls = adls_paths['bronze']
silver_adls = adls_paths['silver']
gold_adls = adls_paths['gold']
# list the files in the ADLS
dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

# url format: https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02
# we need the previous date and the current date to pull from the API url
start_date = date.today() - timedelta(days=1)
end_date = date.today()

# construct the API URL with the start and end date
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    #request to fetch data from the API
    response = requests.get(url)
    # check if the request was successful
    response.raise_for_status()
    # check_for_status() raises an error for bad responses
    data = response.json().get('features', [])
    if not data:
        print("No data returned for the specified date range")
    else:
        # specify the adls path to save data
        adls_path = f"{bronze_adls}/{start_date}_earthquake_data.json"
        # save the data to the adls path
        json_data = json.dumps(data, indent = 4)
        dbutils.fs.put(adls_path, json_data, overwrite = True)
        print(f"Data successfully saved to {adls_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 327227 bytes.
Data successfully saved to abfss://bronze@earthquakedatalake.dfs.core.windows.net//2025-05-02_earthquake_data.json


In [0]:
# define output variables
output_data = {'start_date': start_date.isoformat(),
               'end_date': end_date.isoformat(),
               'bronze_adls': bronze_adls,
               'silver_adls': silver_adls,
               'gold_adls': gold_adls }
dbutils.jobs.taskValues.set(key = 'bronze_output', value = output_data)